In [20]:
import numpy as np
import math

class RamalanInflasi:
    def __init__(self, data):
        self.data = data

    def _hitung_interval(self):
        """Menghitung interval untuk proses fuzzifikasi."""
        min_val = min(self.data)
        max_val = max(self.data)
        rentang_val = max_val - min_val
        ukuran_interval = rentang_val / 3  # Contoh: 3 interval
        interval = [min_val + i * ukuran_interval for i in range(4)]  # 4 batas interval
        return interval

    def _fuzzifikasi(self, nilai, interval):
        """Fuzzifikasi nilai berdasarkan interval."""
        for i in range(len(interval) - 1):
            if interval[i] <= nilai < interval[i + 1]:
                return i
        return len(interval) - 2  # Menetapkan ke interval terakhir jika di luar jangkauan

    def ramalan_markov(self, periode_ramalan=5):
        # Menghitung interval dan fuzzifikasi data
        interval = self._hitung_interval()
        data_fuzzifikasi = [self._fuzzifikasi(val, interval) for val in self.data]

        # Menampilkan interval dan data yang telah difuzzifikasi
        print(f"Interval: {interval}")
        print(f"Data yang telah difuzzifikasi: {data_fuzzifikasi}")

        # Menginisialisasi matriks transisi
        matriks_transisi = np.zeros((len(interval) - 1, len(interval) - 1))

        # Mengisi matriks transisi berdasarkan data yang telah difuzzifikasi
        for i in range(len(data_fuzzifikasi) - 1):
            matriks_transisi[data_fuzzifikasi[i]][data_fuzzifikasi[i + 1]] += 1

        # Normalisasi matriks transisi
        jumlah_baris = matriks_transisi.sum(axis=1, keepdims=True)
        jumlah_baris[jumlah_baris == 0] = 1  # Menghindari pembagian dengan nol
        matriks_transisi /= jumlah_baris

        # Menampilkan matriks transisi
        print(f"Matriks Transisi:\n{matriks_transisi}")

        # Ramalan menggunakan Markov Chain
        ramalan = []
        keadaan_saat_ini = data_fuzzifikasi[-1]

        for _ in range(periode_ramalan):
            keadaan_berikutnya = np.random.choice(len(interval) - 1, p=matriks_transisi[keadaan_saat_ini])
            nilai_ramalan = (interval[keadaan_berikutnya] + interval[keadaan_berikutnya + 1]) / 2
            ramalan.append(nilai_ramalan)
            keadaan_saat_ini = keadaan_berikutnya

        return ramalan

    def ramalan_cheng(self, periode_ramalan=5):
        # Menghitung interval dan fuzzifikasi data
        interval = self._hitung_interval()
        data_fuzzifikasi = [self._fuzzifikasi(val, interval) for val in self.data]

        # Menampilkan interval dan data yang telah difuzzifikasi
        print(f"Interval: {interval}")
        print(f"Data yang telah difuzzifikasi: {data_fuzzifikasi}")

        # Menginisialisasi matriks bobot
        matriks_bobot = np.zeros((len(interval) - 1, len(interval) - 1))

        # Mengisi matriks bobot berdasarkan data yang telah difuzzifikasi
        for i in range(len(data_fuzzifikasi) - 1):
            matriks_bobot[data_fuzzifikasi[i]][data_fuzzifikasi[i + 1]] += 1

        # Normalisasi matriks bobot
        matriks_bobot /= matriks_bobot.sum(axis=1, keepdims=True)

        # Menampilkan matriks bobot
        print(f"Matriks Bobot:\n{matriks_bobot}")

        # Ramalan menggunakan Metode Cheng
        ramalan = []
        keadaan_saat_ini = data_fuzzifikasi[-1]

        for _ in range(periode_ramalan):
            nilai_ramalan = np.sum([
                matriks_bobot[keadaan_saat_ini][j] * (interval[j] + interval[j + 1]) / 2
                for j in range(len(interval) - 1)
            ])
            ramalan.append(nilai_ramalan)
            keadaan_saat_ini = np.argmin(np.abs(
                [(interval[j] + interval[j + 1]) / 2 for j in range(len(interval) - 1)] - nilai_ramalan
            ))

        return ramalan

def main():
    # Data inflasi historis
    data_inflasi = [
        0.0282, 0.0250, 0.0317, 0.0325, 0.0351, 0.0358,
        0.0375, 0.0382, 0.0389, 0.039
    ]
    peramal = RamalanInflasi(data_inflasi)

    # Ramalan menggunakan Markov Chain
    print("Ramalan Markov Chain:")
    ramalan_markov = peramal.ramalan_markov(periode_ramalan=5)
    print(f"Ramalan Inflasi menggunakan Markov Chain: {ramalan_markov}\n")

    # Ramalan menggunakan Metode Cheng
    print("Ramalan Metode Cheng:")
    ramalan_cheng = peramal.ramalan_cheng(periode_ramalan=5)
    print(f"Ramalan Inflasi menggunakan Metode Cheng: {ramalan_cheng}\n")

if __name__ == "__main__":
    main()


Ramalan Markov Chain:
Interval: [0.025, 0.029666666666666668, 0.034333333333333334, 0.039]
Data yang telah difuzzifikasi: [0, 0, 1, 1, 2, 2, 2, 2, 2, 2]
Matriks Transisi:
[[0.5 0.5 0. ]
 [0.  0.5 0.5]
 [0.  0.  1. ]]
Ramalan Inflasi menggunakan Markov Chain: [0.03666666666666667, 0.03666666666666667, 0.03666666666666667, 0.03666666666666667, 0.03666666666666667]

Ramalan Metode Cheng:
Interval: [0.025, 0.029666666666666668, 0.034333333333333334, 0.039]
Data yang telah difuzzifikasi: [0, 0, 1, 1, 2, 2, 2, 2, 2, 2]
Matriks Bobot:
[[0.5 0.5 0. ]
 [0.  0.5 0.5]
 [0.  0.  1. ]]
Ramalan Inflasi menggunakan Metode Cheng: [0.03666666666666667, 0.03666666666666667, 0.03666666666666667, 0.03666666666666667, 0.03666666666666667]

